In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import mplhep as hep
import matplotlib.ticker as mticker
from matplotlib.colors import LogNorm

plt.rcParams.update({"font.size": 16})
plt.style.use(hep.style.CMS)
hep.style.use("CMS")
formatter = mticker.ScalarFormatter(useMathText=True)
formatter.set_powerlimits((-3, 3))

from typing import List
import os

from tqdm import tqdm
import itertools

import warnings

warnings.filterwarnings("ignore", message="invalid value encountered in log10*")

In [ ]:
nonres_scan_cuts = ["txbb", "bdt"]
res_scan_cuts = ["txbb", "thww"]
scan_labels = {"txbb": r"$T_{Xbb}$ WP", "thww": r"$T_{HWW}$ WP", "bdt": "BDT"}

scan_txbb_wps = ["LP", "MP", "HP"]
scan_thww_wps = [0.4, 0.6, 0.8, 0.9, 0.94, 0.96, 0.98]
scan_bdt_wps = [0.6, 0.9, 0.96, 0.99, 0.997, 0.998, 0.999]

plot_dir = "../../../plots/Scans/23May13/"
_ = os.system(f"mkdir -p {plot_dir}")

# cards_dir = "/eos/uscms/store/user/rkansal/bbVV/cards/23May13NonresScan/"
cards_dir = "/uscms/home/rkansal/hhcombine/cards/23May14NonresScan/"

In [ ]:
resonant = False

if resonant:
    scan_wps = list(itertools.product(scan_txbb_wps, scan_thww_wps))
    scan1_wps, scan2_wps = scan_txbb_wps, scan_thww_wps
    scan_cuts = res_scan_cuts

    res_mps = [
        (900, 80),
        (1200, 190),
        (2000, 125),
        (3000, 250),
        (4000, 150),
    ]

    scan_samples = []

    for mX, mY in res_mps:
        scan_samples.append(f"NMSSM_XToYHTo2W2BTo4Q2B_MX-{mX}_MY-{mY}")
else:
    scan_wps = list(itertools.product(scan_txbb_wps, scan_bdt_wps))
    scan1_wps, scan2_wps = scan_txbb_wps, scan_bdt_wps
    scan_cuts = nonres_scan_cuts
    scan_samples = ["HHbbVV"]

In [ ]:
def mxmy(sample):
    mY = int(sample.split("-")[-1])
    mX = int(sample.split("NMSSM_XToYHTo2W2BTo4Q2B_MX-")[1].split("_")[0])

    return (mX, mY)

In [ ]:
limits = {}

# parse limits
for sample in tqdm(scan_samples):
    limits[sample] = {" 2.5": [], "16.0": [], "50.0": [], "84.0": [], "97.5": []}

    for wps in scan_wps:
        cutstr = "_".join([f"{cut}_{wp}" for cut, wp in zip(scan_cuts, wps)])
        sample_dir = "outs" if not resonant else sample
        limits_path = f"{cards_dir}/{cutstr}/{sample_dir}/AsymptoticLimits.txt"

        if os.path.exists(limits_path):
            with open(limits_path, "r") as f:
                lines = f.readlines()

            nums = 0
            for i in np.arange(len(lines) - 1, -1, -1):
                line = lines[i][:-1]
                for key in limits[sample]:
                    start_str = f"Expected {key}%: r < "
                    if line.startswith(start_str):
                        limits[sample][key].append([*wps, float(line.split(start_str)[1])])
                        nums += 1

                if nums == 5:
                    break

            if nums != 5:
                print(f"{cutstr}/{sample} Limits not found!")
        else:
            print(f"{cutstr}/{sample_dir} Limits don't exist!")

In [ ]:
def plot_limits(lim, sample):
    # convert to image
    im = np.zeros((len(scan1_wps), len(scan2_wps)))
    for wp1, wp2, l in lim:
        im[scan1_wps.index(wp1), scan2_wps.index(float(wp2))] = l

    im = im[::-1]

    plt.figure(figsize=(len(scan2_wps) * 2 + 2, 6))
    plt.imshow(im, cmap="turbo")
    if resonant:
        plt.colorbar(label="Expected exclusion limit (fb)")
    else:
        plt.colorbar(label=r"Expected exclusion limit $\times$SM")

    for wp1, wp2, l in lim:
        plt.text(
            scan2_wps.index(float(wp2)),
            len(scan1_wps) - scan1_wps.index(wp1) - 1,
            f"{l:.2f}",
            fontsize=18,
            color="white",
            ha="center",
            va="center",
        )

    plt.xlabel(rf"{scan_labels[scan_cuts[1]]} WP")
    plt.ylabel(rf"{scan_labels[scan_cuts[0]]} WP")

    plt.yticks(np.arange(len(scan1_wps))[::-1], scan1_wps)
    plt.xticks(np.arange(len(scan2_wps)), scan2_wps)

    if resonant:
        mx, my = mxmy(sample)
        plt.title(f"$(M_X, M_Y) = ({mx}, {my})$ GeV", y=1.12)

    hep.cms.label("Work in Progress", data=True, lumi="138.0")

    # plt.title("Signifance Scan")
    plt.savefig(f"{plot_dir}/{sample}_limscan.pdf", bbox_inches="tight")

In [ ]:
for sample, lim in limits.items():
    plot_limits(lim["50.0"], sample)

Old significance scan:

In [ ]:
def to_image(x_range: List[float], y_range: List[float], pixels: np.ndarray):
    """
    ranges: [x_min, x_max (exclusive), # bins]
    """

    im = np.zeros((x_range[2], y_range[2])) - 1  # missing values will show up blank in the image

    for pixel in pixels:
        x = (pixel[0] - x_range[0]) / (x_range[1] - x_range[0]) * x_range[2]
        y = x_range[2] - ((pixel[1] - y_range[0]) / (y_range[1] - y_range[0]) * y_range[2]) - 1
        if x >= x_range[2] or y < 0:
            continue
        im[int(y)][int(x)] = pixel[2] + 1e-12

    return im

In [ ]:
ak8_signs = to_image(BDT_RANGE, BB_RANGE, signs.values[:, :3])
ak8_limits = to_image(
    BDT_RANGE, BB_RANGE, np.concatenate((signs.values[:, :2], signs.values[:, 3:4]), axis=1)
)

In [ ]:
plt.figure(figsize=(16, 12))
plt.imshow(
    ak8_signs,
    extent=[BDT_RANGE[0], BDT_RANGE[1], BB_RANGE[0], BB_RANGE[1]],
    norm=LogNorm(vmin=0.0001, vmax=0.1),
    cmap="jet",
)
plt.colorbar(label="Significance")
plt.xlabel("BDT Cut")
plt.ylabel("Txbb Cut")
plt.title("Signifance Scan")
plt.savefig(f"{plot_dir}/sigscan.pdf", bbox_inches="tight")

In [ ]:
plt.figure(figsize=(16, 12))
plt.imshow(
    ak8_limits,
    extent=[BDT_RANGE[0], BDT_RANGE[1], BB_RANGE[0], BB_RANGE[1]],
    norm=LogNorm(vmin=10, vmax=1000),
    cmap="jet",
)
plt.colorbar(label="Limit")
plt.xlabel("BDT Cut")
plt.ylabel("Txbb Cut")
plt.title("Median xSM Expected Limits")
plt.savefig(f"{plot_dir}/limits.pdf", bbox_inches="tight")